### Multi-attribute swaps

In [2]:
import torch
import argparse

#### First train an autoencoder using the following parameters

```sh
INFO - 05/28/18 19:04:05 - 0:00:00 - ============ Initialized logger ============
INFO - 05/28/18 19:04:05 - 0:00:00 - ae_optimizer: adam,lr=0.0002
                                     ae_reload: 
                                     attr: [('Male', 2), ('Smiling', 2)]
                                     batch_size: 32
                                     clf_dis_reload: 
                                     clip_grad_norm: 5
                                     debug: False
                                     dec_dropout: 0.0
                                     deconv_method: convtranspose
                                     dis_optimizer: adam,lr=0.0002
                                     dump_path: /home/ubuntu/FaderNetworks/models/default/paoucvpr8a
                                     epoch_size: 50000
                                     eval_clf: models/classifier256.pth
                                     h_flip: True
                                     hid_dim: 512
                                     img_fm: 3
                                     img_sz: 256
                                     init_fm: 32
                                     instance_norm: False
                                     lambda_ae: 1
                                     lambda_clf_dis: 0
                                     lambda_lat_dis: 0.0001
                                     lambda_ptc_dis: 0
                                     lambda_schedule: 500000
                                     lat_dis_dropout: 0.3
                                     lat_dis_reload: 
                                     max_fm: 512
                                     n_attr: 4
                                     n_clf_dis: 0
                                     n_epochs: 300
                                     n_lat_dis: 1
                                     n_layers: 6
                                     n_ptc_dis: 0
                                     n_skip: 0
                                     name: default
                                     ptc_dis_reload: 
                                     smooth_label: 0.2
                                     v_flip: False
```

In [12]:
MODEL_PATH = '/home/ubuntu/FaderNetworks/models/default/paoucvpr8a/best_rec_ae.pth'

In [13]:
ae = torch.load(MODEL_PATH).eval()

/home/ubuntu/anaconda3/lib/python3.6/site-packages/torch/serialization.py:325: SourceChangeWarning: source code of class 'torch.nn.modules.conv.Conv2d' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)


In [14]:
from src.logger import create_logger
from src.loader import load_images, DataSampler
from src.utils import bool_flag

In [15]:
parser = argparse.ArgumentParser(description='Attributes swapping')
parser.add_argument("--model_path", type=str, default="",
                    help="Trained model path")
parser.add_argument("--n_images", type=int, default=10,
                    help="Number of images to modify")
parser.add_argument("--offset", type=int, default=0,
                    help="First image index")
parser.add_argument("--n_interpolations", type=int, default=10,
                    help="Number of interpolations per image")
parser.add_argument("--alpha_min", type=float, default=1,
                    help="Min interpolation value")
parser.add_argument("--alpha_max", type=float, default=1,
                    help="Max interpolation value")
parser.add_argument("--plot_size", type=int, default=5,
                    help="Size of images in the grid")
parser.add_argument("--row_wise", type=bool_flag, default=True,
                    help="Represent image interpolations horizontally")
parser.add_argument("--output_path", type=str, default="output.png",
                    help="Output path")

_StoreAction(option_strings=['--output_path'], dest='output_path', nargs=None, const=None, default='output.png', type=<class 'str'>, choices=None, help='Output path', metavar=None)

#### Simulating argparse in command line

In [16]:
argv = '--model_path models/default/paoucvpr8a/best_rec_ae.pth --n_images 10 --n_interpolations 10 --alpha_min 2.0 --alpha_max 2.0 --output_path test_eyes.png'.split( )

In [17]:
params = parser.parse_args(argv)

In [18]:
params

Namespace(alpha_max=2.0, alpha_min=2.0, model_path='models/default/paoucvpr8a/best_rec_ae.pth', n_images=10, n_interpolations=10, offset=0, output_path='test_eyes.png', plot_size=5, row_wise=True)

#### Restore default values for some parameters

In [19]:
params.debug = True
params.batch_size = 32
params.v_flip = False
params.h_flip = False
params.img_sz = ae.img_sz
params.attr = ae.attr
params.n_attr = ae.n_attr

In [20]:
for (i, n_cat) in params.attr:
    print(i, n_cat)

Male 2
Smiling 2


#### Load dataset

In [23]:
import os

In [24]:
DATA_PATH = '/home/ubuntu/FaderNetworks/data'
images_filename = 'images_%i_%i.pth'
images_filename = images_filename % (params.img_sz, params.img_sz)

In [25]:
images = torch.load(os.path.join(DATA_PATH, images_filename))

In [26]:
images.shape

torch.Size([202599, 3, 256, 256])

In [27]:
attributes = torch.load(os.path.join(DATA_PATH, 'attributes.pth'))

#### parse attributes

In [29]:
import numpy as np

attrs = []
for name, n_cat in params.attr:
    for i in range(n_cat):
        attrs.append(torch.FloatTensor((attributes[name] == i).astype(np.float32)))
        
attributes = torch.cat([x.unsqueeze(1) for x in attrs], 1)

In [30]:
attributes.shape

torch.Size([202599, 4])

In [31]:
train_index = 10000
valid_index = 15000
test_index = 20000

In [32]:
train_images = images[:train_index]
valid_images = images[train_index:valid_index]
test_images = images[valid_index:test_index]
train_attributes = attributes[:train_index]
valid_attributes = attributes[train_index:valid_index]
test_attributes = attributes[valid_index:test_index]

In [33]:
data = train_images, valid_images, test_images
attributes = train_attributes, valid_attributes, test_attributes

In [34]:
test_data = DataSampler(data[2], attributes[2], params)

In [35]:
from torch.autograd import Variable

### Reconstruct images / create interpolations



In [98]:
test_data.images.size(0)
#images, attributes = test_data.eval_batch

5000

In [99]:
images, attributes = test_data.eval_batch(40, 50)

In [100]:
enc_outputs = ae.encode(images)

In [101]:
# interpolation values
alphas = np.linspace(1 - params.alpha_min, params.alpha_max, params.n_interpolations)


In [102]:
alphas

array([-1.        , -0.66666667, -0.33333333,  0.        ,  0.33333333,
        0.66666667,  1.        ,  1.33333333,  1.66666667,  2.        ])

In [103]:
alphas = [torch.FloatTensor([1 - alpha, alpha, 1 - alpha, alpha]) for alpha in alphas]

In [104]:
alphas

[
  2
 -1
  2
 -1
 [torch.FloatTensor of size 4], 
  1.6667
 -0.6667
  1.6667
 -0.6667
 [torch.FloatTensor of size 4], 
  1.3333
 -0.3333
  1.3333
 -0.3333
 [torch.FloatTensor of size 4], 
  1
  0
  1
  0
 [torch.FloatTensor of size 4], 
  0.6667
  0.3333
  0.6667
  0.3333
 [torch.FloatTensor of size 4], 
  0.3333
  0.6667
  0.3333
  0.6667
 [torch.FloatTensor of size 4], 
  0
  1
  0
  1
 [torch.FloatTensor of size 4], 
 -0.3333
  1.3333
 -0.3333
  1.3333
 [torch.FloatTensor of size 4], 
 -0.6667
  1.6667
 -0.6667
  1.6667
 [torch.FloatTensor of size 4], 
 -1
  2
 -1
  2
 [torch.FloatTensor of size 4]]

In [105]:
# original image / reconstructed image / interpolations
outputs = []
outputs.append(images)

In [106]:
outputs.append(ae.decode(enc_outputs, attributes)[-1])

In [107]:
enc_outputs[0].size(0)

10

In [108]:
params.n_attr

4

### What is going on inside the for-loop

In [109]:
alpha = alphas[0]

In [110]:
alpha


 2
-1
 2
-1
[torch.FloatTensor of size 4]

In [111]:
alpha.unsqueeze(0)


 2 -1  2 -1
[torch.FloatTensor of size 1x4]

In [112]:
Variable(alpha.unsqueeze(0).expand((10,alpha.size(0)))).cuda

<bound method Variable.cuda of Variable containing:
    2    -1     2    -1
    2    -1     2    -1
    2    -1     2    -1
    2    -1     2    -1
    2    -1     2    -1
    2    -1     2    -1
    2    -1     2    -1
    2    -1     2    -1
    2    -1     2    -1
    2    -1     2    -1
[torch.FloatTensor of size 10x4]
>

### Decoder outputs

In [113]:
for alpha in alphas:
    alpha = Variable(alpha.unsqueeze(0).expand((len(images), alpha.size(0))).cuda())
    outputs.append(ae.decode(enc_outputs, alpha)[-1])

In [114]:
result = torch.cat([x.unsqueeze(1) for x in outputs], 1).data.cpu()

In [115]:
result.shape

torch.Size([10, 12, 3, 256, 256])

In [116]:
interpolations = torch.cat(result, 0)

In [117]:
interpolations.size()

torch.Size([120, 3, 256, 256])

In [118]:
params.n_images, 2 + params.n_interpolations, 3, params.img_sz, params.img_sz


(10, 12, 3, 256, 256)

In [119]:
assert result.size() == (params.n_images, 2 + params.n_interpolations,
                                 3, params.img_sz, params.img_sz)

In [120]:
from torchvision.utils import make_grid

In [121]:
def get_grid(images, row_wise, plot_size=5):
    """
    Create a grid with all images.
    """
    n_images, n_columns, img_fm, img_sz, _ = images.size()
    if not row_wise:
        images = images.transpose(0, 1).contiguous()
    images = images.view(n_images * n_columns, img_fm, img_sz, img_sz)
    images.add_(1).div_(2.0)
    return make_grid(images, nrow=(n_columns if row_wise else n_images))


In [122]:
grid = get_grid(result, params.row_wise, params.plot_size)

In [123]:
params.output_path = 'gender_smile.png'

In [124]:
import matplotlib.image

In [125]:
matplotlib.image.imsave(params.output_path, grid.numpy().transpose((1, 2, 0)))